In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import pandas as pd
import os
import face_recognition
from keras_facenet import FaceNet
from matplotlib import pyplot as plt
#from keras.models import load_model
#from PIL import Image

In [ ]:
model = YOLO('best.pt')

embedder = FaceNet()
#cap = cv2.VideoCapture(0)

def filePaths(directory):
    files = []
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            files.append(os.path.join(dirpath, f))
    return files

def one_hot(array):
    unique, inverse = np.unique(array, return_inverse=True)
    onehot = np.eye(unique.shape[0])[inverse]
    return onehot

X = []
y = []

for filename in filePaths('Class_Data'):
    frame = cv2.imread(filename)
    results = model(frame)
    b = results[0].boxes.xyxy[0].numpy().astype(int)
    print(b)
    annotated_frame = frame[b[1]:b[3], b[0]:b[2]]
    face_image = cv2.resize(annotated_frame, (128, 128))
    embedding_result = embedder.embeddings([face_image])[0]
    X.append(embedding_result)
    y.append(filename[filename.find('\\')+1:filename.find('\\IMG')])
    cv2.imshow("YOLOv8 Inference", face_image)

unique = list(np.unique(y))
y = np.array([unique.index(elem) for elem in y])

In [ ]:
X_test = []
y_test = []

frame = cv2.imread('test1.jpg')
results = model(frame)
b = results[0].boxes.xyxy[0].numpy().astype(int)
annotated_frame = frame[b[1]:b[3], b[0]:b[2]]
face_image = cv2.resize(annotated_frame, (128, 128))
embedding_result = embedder.embeddings([face_image])[0]
X_test.append(embedding_result)
y_test.append('Pranav Kuppa')

frame = cv2.imread('test2.jpg')
results = model(frame)
b = results[0].boxes.xyxy[0].numpy().astype(int)
annotated_frame = frame[b[1]:b[3], b[0]:b[2]]
face_image = cv2.resize(annotated_frame, (128, 128))
embedding_result = embedder.embeddings([face_image])[0]
X_test.append(embedding_result)
y_test.append('Pranav Kuppa')

frame = cv2.imread('test3.jpg')
results = model(frame)
b = results[0].boxes.xyxy[0].numpy().astype(int)
annotated_frame = frame[b[1]:b[3], b[0]:b[2]]
face_image = cv2.resize(annotated_frame, (128, 128))
embedding_result = embedder.embeddings([face_image])[0]
X_test.append(embedding_result)
y_test.append('Ayaan Siddiqui')

y_test = np.array([unique.index(elem) for elem in y_test])

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
pca.fit(X)
X_pca=pca.transform(X)

pca=PCA()
pca.fit(X)

plt.figure(1, figsize=(12,8))

plt.plot(pca.explained_variance_, linewidth=2)
 
plt.xlabel('Components')
plt.ylabel('Explained Variaces')
plt.show()

In [ ]:
n_components = 50

pca=PCA(n_components=n_components, whiten=True)
pca.fit(X)

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics

X_pca=pca.transform(X)
X_test_pca = pca.transform(X_test)

In [ ]:
X_pca.shape

In [ ]:
X_test_pca.shape

In [ ]:
y.shape

In [ ]:
np.array([np.argmax(a) for a in y]).shape

In [ ]:
clf = SVC(kernel='linear', probability=True)

clf.fit(X_pca, y)

y_pred = clf.predict(X_test_pca)

print(y_test)
print(y_pred)

In [ ]:
import pickle

with open('model.pkl','wb') as f:
    pickle.dump(clf,f)

In [ ]:
with open('pca_model.pkl','wb') as f:
    pickle.dump(pca,f)